<a href="https://colab.research.google.com/github/PriyankaMittapelly/SaiPriyanka_INFO5731_Spring2020/blob/main/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [ ]:
import nltk, re, string, collections
from nltk.util import ngrams

with open("/content/Abstract1.csv", "r", encoding='latin-1') as file:
    text = file.read()

tokenized = text.split()

# and get a list of all the tri-grams
esBigrams = ngrams(tokenized, 3)

esBigramFreq = collections.Counter(esBigrams)

Trigram = dict(esBigramFreq)

print(Trigram)

{('Abstract', 'Abstract', 'not'): 1, ('Abstract', 'not', 'found'): 8, ('not', 'found', '"'): 4, ('found', '"', 'describe'): 1, ('"', 'describe', 'a'): 1, ('describe', 'a', 'method'): 1, ('a', 'method', 'for'): 2, ('method', 'for', 'statistical'): 1, ('for', 'statistical', 'modeling'): 1, ('statistical', 'modeling', 'based'): 1, ('modeling', 'based', 'on'): 1, ('based', 'on', 'maximum'): 1, ('on', 'maximum', 'entropy.'): 1, ('maximum', 'entropy.', 'We'): 1, ('entropy.', 'We', 'present'): 1, ('We', 'present', 'a'): 2, ('present', 'a', 'maximum-likelihood'): 1, ('a', 'maximum-likelihood', 'approach'): 1, ('maximum-likelihood', 'approach', 'for'): 1, ('approach', 'for', 'automatically'): 1, ('for', 'automatically', 'constructing'): 1, ('automatically', 'constructing', 'maximum'): 1, ('constructing', 'maximum', 'entropy'): 1, ('maximum', 'entropy', 'models'): 1, ('entropy', 'models', 'and'): 1, ('models', 'and', 'describe'): 1, ('and', 'describe', 'how'): 1, ('describe', 'how', 'to'): 1, ('

In [ ]:
from __future__ import division
import nltk
import re

f = open('/content/Abstractfile.csv', 'r')
bigramfile = f.read()
from nltk.tokenize import sent_tokenize
sent_tokenize_list = sent_tokenize(bigramfile)
print("count of w2,w1/w1")
for i in sent_tokenize_list:
    tokens = i.split()
    bigrams = (tuple(nltk.bigrams(tokens)))
    bigrams_frequency = nltk.FreqDist(bigrams)
    for key , value  in bigrams_frequency.items():
        
        unigrams = tokens.count(key[0])
        final_output = value / unigrams
        print('{}  : {}' .format(key , final_output))

count of w2,w1/w1
('Abstract', 'not')  : 1.0
('not', 'found')  : 1.0
('found', '"')  : 1.0
('"', 'describe')  : 1.0
('describe', 'a')  : 1.0
('a', 'method')  : 1.0
('method', 'for')  : 1.0
('for', 'statistical')  : 1.0
('statistical', 'modeling')  : 1.0
('modeling', 'based')  : 1.0
('based', 'on')  : 1.0
('on', 'maximum')  : 1.0
('maximum', 'entropy.')  : 1.0
('We', 'present')  : 1.0
('present', 'a')  : 1.0
('a', 'maximum-likelihood')  : 1.0
('maximum-likelihood', 'approach')  : 1.0
('approach', 'for')  : 0.5
('for', 'automatically')  : 1.0
('automatically', 'constructing')  : 1.0
('constructing', 'maximum')  : 1.0
('maximum', 'entropy')  : 1.0
('entropy', 'models')  : 1.0
('models', 'and')  : 1.0
('and', 'describe')  : 1.0
('describe', 'how')  : 1.0
('how', 'to')  : 1.0
('to', 'implement')  : 1.0
('implement', 'this')  : 1.0
('this', 'approach')  : 1.0
('approach', 'efficiently,')  : 0.5
('efficiently,', 'using')  : 1.0
('using', 'as')  : 1.0
('as', 'examples')  : 1.0
('examples', 'se

In [ ]:
#NOUN FREQUENCIES 
import spacy
Ndata = spacy.load("en")
filename = open("Abstract1.csv","r")
File = Ndata(filename.read())


n = []

for i in File.noun_chunks:
  n.append(i.text)
df = pd.DataFrame(n, columns=['NOUNPHRASES'])
word_vectorizer = CountVectorizer(ngram_range=(3,3), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(df['NOUNPHRASES'].values.astype('U'))
frequency = sum(sparse_matrix).toarray()[0]
df1 = pd.DataFrame(frequency, index=word_vectorizer.get_feature_names(),columns=['Frequency'])
df1['NounProbabilities'] = df1/df1.max()
print(n)
df1

['Abstract\n                     Abstract', 'a method', 'statistical modeling', 'maximum entropy', 'We', 'a maximum-likelihood approach', 'maximum entropy models', 'this approach', 'examples', 'several problems', 'natural language processing', 'conditional random fields', 'natural language processing Terms', 'Conditions', 'Terms', 'Conditions', 'Copyright', 'works', 'Minerva Access', 'The paper', 'the issue', 'cooperation', 'linguistics', 'natural language processing', 'NLP', 'linguistics', 'machine translation', '(MT', 'It', 'just one direction', 'such cooperation', 'namely applications', 'linguistics', 'NLP', 'most natural language processing applications', 'Description Logics', 'a knowledge base', 'some syntactic, semantic, and pragmatic elements', 'the semantic interpretation', 'the natural language generation', 'Description Logics', 'We', 'a unified neural network architecture', 'learning algorithm', 'various natural language processing tasks', 'speech', 'semantic role labeling', 

,Frequency,NounProbabilities
1980s nlp research,1,0.013889
21 user callable,1,0.013889
36 traditional approaches,1,0.013889
abstract many information,1,0.013889
abstract natural language,2,0.027778
...,...,...
web scale data,1,0.013889
word based document,1,0.013889
word sense disambiguating,1,0.013889
word sense disambiguation,1,0.013889


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [ ]:

#**********************************************tf-idf


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import csv
import pandas as pd

posdata = []
with open('/content/Abstract1.csv' , 'r') as input:
  reader = csv.reader(input)
  for row in reader:
    posdata.extend(row)
vectorizer = TfidfVectorizer()
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
for i , tvar  in zip(range(500), posdata):

  #new = input[tvar]
  #print(tvar)
  tfIdf = tfIdfVectorizer.fit_transform([tvar])
  df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
  df = df.sort_values('TF-IDF', ascending=False)
  print(df)



          TF-IDF
abstract     1.0
           TF-IDF
abstract  0.57735
found     0.57735
not       0.57735
                 TF-IDF
maximum        0.420084
for            0.280056
approach       0.280056
describe       0.280056
entropy        0.280056
and            0.140028
natural        0.140028
on             0.140028
present        0.140028
problems       0.140028
processing     0.140028
modeling       0.140028
several        0.140028
statistical    0.140028
this           0.140028
to             0.140028
using          0.140028
models         0.140028
likelihood     0.140028
method         0.140028
language       0.140028
in             0.140028
implement      0.140028
how            0.140028
examples       0.140028
efficiently    0.140028
constructing   0.140028
based          0.140028
automatically  0.140028
as             0.140028
we             0.140028
               TF-IDF
terms        0.348155
conditions   0.348155
in           0.348155
and          0.348155
access       0.1

In [ ]:
#****************cosine similarity.


import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords 

from nltk.tokenize import word_tokenize 

  
# X = input("Enter first string: ").lower() 
# Y = input("Enter second string: ").lower() 

X ="hello ! how are you ?"

Y ="I'm good , thanks . How are you doing?"

  
# tokenization 

X_list = word_tokenize(X)  

Y_list = word_tokenize(Y) 

  
# sw contains the list of stopwords 

sw = stopwords.words('english')  

l1 =[];l2 =[] 

  
# remove stop words from the string 

X_set = {w for w in X_list if not w in sw}  

Y_set = {w for w in Y_list if not w in sw} 

  
# form a set containing keywords of both strings  

rvector = X_set.union(Y_set)  

for w in rvector: 

    if w in X_set: l1.append(1) # create a vector 

    else: l1.append(0) 

    if w in Y_set: l2.append(1) 

    else: l2.append(0) 

c = 0

  
# cosine formula  

for i in range(len(rvector)): 

        c+= l1[i]*l2[i] 

cosine = c / float((sum(l1)*sum(l2))*0.5) 

print(" cosine similarity: ", cosine)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
 cosine similarity:  0.08333333333333333


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
import nltk
import pandas as pd 
from nltk.corpus  import  stopwords

from nltk.stem import PorterStemmer

import string

from textblob import TextBlob , Word 

data =  "/content/Abstract1.csv"

word = pd.read_csv(data)
#ALL CHAR TO LOWER CASE
word['Abstract'] = word['Abstract'].apply(lambda x :" ".join(x.lower() for x in x.split()))
word['Abstract']
type1 = pd.DataFrame(word)


# STOP WORDS REMOVAL
nltk.download('stopwords')
stop = stopwords.words('english')

type1['Abstract'] = type1['Abstract'].apply(lambda x :" ".join([x for x in  x.split() if x not in stop ]))
type2 = pd.DataFrame(type1)

#punctuation

type2['Abstract'] = type2['Abstract'].apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))
type3 = pd.DataFrame(type2)

#FREQUENT WORDS REMOVAL

fre = pd.Series(' '.join(type2['Abstract']).split()).value_counts()[:20]
fre = list(fre.index)
type3['Abstract'] = type3['Abstract'].apply(lambda x :" ".join([x for x in  x.split() if x not in fre ]))
type4 = pd.DataFrame(type3)

# RARE WORDS REMOVAL

'''rar = pd.Series(' '.join(type2['Abstract']).split()).value_counts()[-20:]
rar = list(rar.index)
type4['Abstract'] = type4['Abstract'].apply(lambda x :" ".join([x for x in  x.split() if x not in rar ]))
type5 = pd.DataFrame(type4)
type5 '''

# SPELLING CORRECTION 


type4['Abstract'] = type4['Abstract'] .apply(lambda x : str(TextBlob(x).correct()))
type6 = pd.DataFrame(type4)





# STEMMING 
st = PorterStemmer()
type6['Abstract'] = type5['Abstract'].apply(lambda x : " ".join([st.stem(info) for info in x.split()]))
type7 = pd.DataFrame(type6)


#lemmatization

nltk.download('wordnet')
type7['DATAafter cleaning'] = type6['Abstract'].apply(lambda x : " ".join([Word(values).lemmatize() for values in x.split()]))
df1 = pd.DataFrame(type7)


# all data after cleaning , storing in new datacleaning colum 

df1.to_csv('datacleaningg.csv', index=True)

#SENTMENTALANALYSISFILE IS ATTACHED WHERE SENTIEMNT ANALYSIS IS MENTIONED IN SEPERATE COLUMN


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
